In [1]:
import pandas as pd

In [2]:
#주어진 개수 인수에 따라 train, dev, test dataset을 나누는 기능. 

In [3]:
#데이터 내 라벨 분포 확인하기 위한 함수
def countLabel(data):
    class0 = sum(data['label']==0)
    class1 =  sum(data['label']==1)
    class2 =  sum(data['label']==2)
    
    return class0, class1, class2

In [4]:
def split_data(df, train_num, dev_num, test_num, uneven=False):
    
    seed=100
    class0, class1, class2 = countLabel(df)
    
    """test data set"""
    remain_df = df.copy()
    df0 = df.loc[(df['label']==0)].sample(n = test_num, random_state = seed) 
    remain_df = remain_df.drop(df0.index)
    
    df1 = df.loc[(df['label']==1)].sample(n = test_num, random_state = seed) 
    remain_df = remain_df.drop(df1.index)

    df2 = df.loc[(df['label']==2)].sample(n = test_num, random_state = seed) 
    remain_df = remain_df.drop(df2.index)

    remain_df.reset_index(drop=True, inplace=True)
    test_df = pd.DataFrame()
    test_df = df0.append(df1).append(df2)
    test_df.reset_index(drop=True, inplace=True)
    
    """dev data set"""
    df0=remain_df.loc[(remain_df['label']==0)].sample(n = dev_num, random_state = seed) 
    remain_df = remain_df.drop(df0.index)
   
    df1=remain_df.loc[(remain_df['label']==1)].sample(n = dev_num, random_state = seed) 
    remain_df = remain_df.drop(df1.index)

    df2=remain_df.loc[(remain_df['label']==2)].sample(n = dev_num, random_state = seed) 
    remain_df = remain_df.drop(df2.index)
   
    remain_df.reset_index(drop=True, inplace=True)
    eval_df = pd.DataFrame()
    eval_df = df0.append(df1).append(df2)
    eval_df.reset_index(drop=True, inplace=True)
    
    """train data set"""
    train_df = remain_df.copy()   
    if uneven == True: return train_df, eval_df, test_df
    
    sub0, sub1, sub2 = countLabel(remain_df)
    d_n=[]
    for sub in [sub0, sub1, sub2]:
        if train_num>sub:
            d_n.append(sub)
        else : d_n.append(train_num)
    
    df0=remain_df.loc[(remain_df['label']==0)].sample(n = d_n[0], random_state = seed) 
    df1=remain_df.loc[(remain_df['label']==1)].sample(n = d_n[1], random_state = seed) 
    df2=remain_df.loc[(remain_df['label']==2)].sample(n = d_n[2], random_state = seed) 
  
    train_df = pd.DataFrame()
    train_df = df0.append(df1).append(df2)
    
    #모자란 개수 upsampling 
    for index, num in enumerate(d_n):
        sub = train_num-num
        if sub>0:
            #print(train_num-num)
            over_df = remain_df.loc[(remain_df['label']==index)].sample(n = sub, random_state = seed)
            train_df = train_df.append(over_df)
            print("we add %d to train_df."%(sub))
        
    train_df.reset_index(drop=True, inplace=True)

    return train_df, eval_df, test_df


In [5]:
def make_file(train_df, dev_df, test_df, data_dir): 
    train_df.to_csv(data_dir  + 'train.tsv', index=False, header=True, sep='\t')
    dev_df.to_csv(data_dir  +  'dev.tsv', index=False, header=True, sep='\t')
    test_df.to_csv(data_dir +  'test.tsv', index=False, header=True, sep='\t')

In [6]:
#main
def main(data_dir, data_file): #입력 데이터 이름 
    data = pd.read_csv(data_dir+data_file, sep='\t') #data의 header는 text, label 두 가지로 이루어져 있다.
    #uneven - even
    train1, dev1, test1 = split_data(data, 0,12,25, True)
    make_file(train1, dev1, test1, data_dir+'uneven-even/')
    #even - even
    train2, dev2, test2 = split_data(data, 90, 12, 25, False)
    make_file(train2, dev2, test2, data_dir+'even-even/')
    #(up)even - even
    train3, dev3, test3 = split_data(data, 145,12,25, False)
    make_file(train3, dev3, test3, data_dir+'upeven-even/')
    

In [7]:
main("../data/" ,'Thyroid_bind.tsv')

we add 55 to train_df.


In [7]:
import pandas as pd

data = pd.read_csv("../data/Thyroid_bind.tsv", sep='\t') 
class0, class1, class2 = countLabel(data)
total = class0+ class1+ class2
print(total, class0/total*100, class1/total*100, class2/total*100)

1307 76.35807192042846 13.925019127773528 9.716908951798011
